# 残差网络

欢迎来到本周的第一个任务！您将构建一个非常深的卷积网络，使用残差网络（ResNets）。理论上，非常深的网络可以表示非常复杂的函数; 但在实践中，它们很难训练。由[He等人介绍的残差网络]（https://arxiv.org/pdf/1512.03385.pdf）允许您训练比以前可行的更深的网络。

在本任务结束时，您将能够：

- 使用Keras在深度神经网络中实现ResNets的基本构建块

- 将这些构建块组合在一起，实现和训练用于图像分类的最先进的神经网络

- 在您的网络中实现跳过连接

对于此任务，您将使用Keras。

在进入问题之前，请运行下面的单元格以加载所需的软件包。

## 关于提交到自动分级器的重要说明

在将您的作业提交到自动分级器之前，请确保您没有执行以下任何操作：

1.您没有在任务中添加任何_extra_ `print`语句。

2.您没有在任务中添加任何_extra_代码单元格。

3.您没有更改任何函数参数。

4.您没有在评分练习中使用任何全局变量。除非特别要求这样做，请避免使用它并改用本地变量。

5.您没有在不需要的情况下更改任务代码，例如创建_extra_变量。

如果您执行了以下任何操作，则在提交作业时将获得类似于“分级器错误：找不到分级器反馈”（或类似意外的）错误。在请求帮助/调试作业中的错误之前，请首先检查这些内容。如果是这种情况，并且您不记得您所做的更改，则可以按照以下[说明]（https://www.coursera.org/learn/convolutional-neural-networks/supplement/DS4yP/h-ow-to-refresh-your-workspace）获取任务的新副本。

## Table of Content

- [1 - Packages](#1)
- [2 - The Problem of Very Deep Neural Networks](#2)
- [3 - Building a Residual Network](#3)
    - [3.1 - The Identity Block](#3-1)
        - [Exercise 1 - identity_block](#ex-1)
    - [3.2 - The Convolutional Block](#3-2)
        - [Exercise 2 - convolutional_block](#ex-2)
- [4 - Building Your First ResNet Model (50 layers)](#4)
    - [Exercise 3 - ResNet50](#ex-3)
- [5 - Test on Your Own Image (Optional/Ungraded)](#5)
- [6 - Bibliography](#6)

<a name='1'></a>
## 1 - Packages

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

from test_utils import summary, comparator
import public_tests

%matplotlib inline

<a name='2'></a>
## 2 - 非常深的神经网络问题

上周，您使用numpy手动构建了第一个卷积神经网络，然后使用Tensorflow和Keras进行了构建。

近年来，神经网络变得更加深层，最先进的网络从仅具有少数几层（例如AlexNet）发展到超过100层。

* 非常深的网络的主要优点是可以表示非常复杂的函数。 它还可以在许多不同抽象层面上学习特征，从边缘（在较浅的层，接近输入）到非常复杂的特征（在较深的层，接近输出）。

* 然而，使用更深的网络并不总是有帮助的。 训练它们的一个巨大障碍是消失的梯度：非常深的网络通常具有很快变为零的梯度信号，因此使梯度下降变得极其缓慢。

* 更具体地说，在梯度下降期间，当您从最后一层向后传播到第一层时，您在每一步中都会乘以权重矩阵，因此梯度可以指数级地快速减少到零（或在极少数情况下呈指数级增长并“爆炸”，从而获得非常大的值）。

* 在训练期间，因此您可能会看到浅层梯度的大小（或范数）随着训练的进行而非常快速地减少，如下所示：

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><font color='purple'>  : <b>Vanishing gradient</b> <br> The speed of learning decreases very rapidly for the shallower layers as the network trains </center></caption>

Not to worry! You are now going to solve this problem by building a Residual Network!

<a name='3'></a>
## 3 - 构建残差网络

在ResNets中，“shortcut”或“skip connection”允许模型跳过层：

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> <b>图2</b> </u><font color='purple'>  ：显示具有skip-connection的ResNet块 <br> </center></caption>






左侧的图像显示了网络的“主路径”。右侧的图像添加了主路径的shortcut。通过将这些ResNet块堆叠在一起，可以形成非常深的网络。

讲座提到，具有shortcut的ResNet块也使得其中一个块很容易学习identity function。这意味着您可以堆叠额外的ResNet块，而几乎不会危及训练集性能。

在这一点上，也有一些证据表明，学习identity function的易度比skip connections有助于解决梯度消失，更能解释ResNets的出色性能。

ResNet中使用了两种主要类型的块，主要取决于输入/输出尺寸是否相同。您将实现它们两个： "identity block"和 "convolutional block"。

<a name='3-1'></a>
### 3.1 - 恒等块

恒等块是ResNets中使用的标准块，对应于输入激活（$a^{[l]}$）与输出激活（$a^{[l+2]}$）具有相同维度的情况。为了阐明ResNet的恒等块中发生的不同步骤，这里提供了一个显示各个步骤的替代图：

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">

<caption><center> <u> <font color='purple'> <b>图3</b> </u><font color='purple'> : <b>恒等块。</b> 跳跃连接“跳过”2层。 </center></caption>

上面的路径是“捷径路径”。下面的路径是“主路径”。在此图中，注意每层中的CONV2D和ReLU步骤。为了加快训练速度，已添加了BatchNorm步骤。不必担心这很难实现--您将看到，在Keras中，BatchNorm只需要一行代码！

在此练习中，您将实际实现这个恒等块的略微更强大的版本，其中跳过连接“跳过”3个隐藏层而不是2个层。它看起来像这样：

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">

<caption><center> <u> <font color='purple'> <b>图4</b> </u><font color='purple'> : <b>恒等块。</b> 跳跃连接“跳过”3层。</center></caption>

这些是单个步骤：

主路径的第一个组件：
- 第一个CONV2D具有形状为（1,1）且步幅为（1,1）的$F_1$个滤波器。其填充为“valid”。使用0作为随机均匀初始化的种子：`kernel_initializer = initializer(seed=0)`。
- 第一个BatchNorm对“channels”轴进行归一化。
- 然后应用ReLU激活函数。这没有超参数。

主路径的第二个组件：
- 第二个CONV2D具有形状为$(f,f)$且步幅为（1,1）的$F_2$个滤波器。其填充为“same”。使用0作为随机均匀初始化的种子：`kernel_initializer = initializer(seed=0)`。
- 第二个BatchNorm对“channels”轴进行归一化。
- 然后应用ReLU激活函数。这没有超参数。

主路径的第三个组件：
- 第三个CONV2D具有形状为（1,1）且步幅为（1,1）的$F_3$个滤波器。其填充为“valid”。使用0作为随机均匀初始化的种子：`kernel_initializer = initializer(seed=0)`。
- 第三个BatchNorm对“channels”轴进行归一化。
- 请注意，此组件中**没有**ReLU激活函数。

最后一步：
- `X_shortcut`和第3层的输出`X`加在一起。
- **提示**：语法看起来像`Add()([var1,var2])`
- 然后应用ReLU激活函数。这没有超参数。
<a name='ex-1'></a>
### 练习1 - identity_block

实现ResNet恒等块。主路径的第一个组件已经为您实现！首先，您应该仔细阅读这些文档，以确保您理解正在发生的事情。然后，实现其余部分。
- 要实现Conv2D步骤：[Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- 要实现BatchNorm：[BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)`BatchNormalization(axis = 3)(X, training = training)`。如果将training设置为False，则其权重不会随新示例更新。也就是说，当模型在预测模式下使用时。
- 对于激活，请使用：`Activation（'relu'）（X）`
- 要添加由快捷方式传递的值：[Add](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

我们已向我们的函数添加了initializer参数。此参数接收像[tensorflow.keras.initializers](https://www.tensorflow.org/api_docs/python/tf/keras/initializers)包中包含的函数一样的初始化器函数，或任何其他自定义初始化器。默认情况下，它将设置为[random_uniform](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform)

请记住，这些函数接受一个`seed`参数，其值可以是您想要的任何值，但在此笔记本中必须设置为0，以进行**分级**。

 Here is where you're actually using the power of the Functional API to create a shortcut path: 

In [ ]:
# UNQ_C1
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X) 
    ### END CODE HERE

    return X

In [ ]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

public_tests.identity_block_test(identity_block)

**Expected value**

```
With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71234 192.71234 192.71234  96.85617]
  [ 96.85617  96.85617  96.85617  48.92808]]

 [[578.1371  578.1371  578.1371  290.5685 ]
  [290.5685  290.5685  290.5685  146.78426]]]
96.85617

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
```

<a name='3-2'></a>

### 3.2 - 卷积块

ResNet的“卷积块”是第二种块类型。当输入和输出的维数不匹配时，可以使用这种类型的块。与恒等块的区别在于快捷路径中有一个CONV2D层：

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">

<caption><center> <u> <font color='purple'> <b>图4</b> </u><font color='purple'> : <b>卷积块</b> </center></caption>


* 快捷路径中的CONV2D层用于调整输入$x$的尺寸到不同的维度，以便在最终加法中将快捷值添加回主路径中时维度匹配。 （这在讲座中讨论的矩阵$W_s$中扮演类似的角色。）
* 例如，为了将激活维度的高度和宽度减小2倍，可以使用步长为2的1x1卷积。
* 快捷路径上的CONV2D层不使用任何非线性激活函数。它的主要作用只是应用（学习的）线性函数，将输入的维度减小，以便在后续加法步骤中维度匹配。
* 至于以前的练习，额外的`initializer`参数是为了评分而需要的，并且已经默认设置为[glorot_uniform]（https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform）

卷积块的详细信息如下。

主路径的第一个组件：

- 第一个CONV2D具有形状为（1,1）的$F_1$个滤波器和（s，s）的步幅。其填充为“valid”。使用0作为“glorot_uniform”种子“kernel_initializer = initializer（seed = 0）”的值。

- 第一个BatchNorm对“channels”轴进行归一化。

- 然后应用ReLU激活函数。这没有超参数。

主路径的第二个组件：

- 第二个CONV2D具有形状为（f，f）的$F_2$个滤波器和（1,1）的步幅。其填充为“same”。使用0作为“glorot_uniform”种子“kernel_initializer = initializer（seed = 0）”的值。

- 第二个BatchNorm对“channels”轴进行归一化。

- 然后应用ReLU激活函数。这没有超参数。

主路径的第三个组件：

- 第三个CONV2D具有形状为（1,1）的$F_3$个滤波器和（1,1）的步幅。其填充为“valid”。使用0作为“glorot_uniform”种子“kernel_initializer = initializer（seed = 0）”的值。

- 第三个BatchNorm对“channels”轴进行归一化。请注意，此组件中没有ReLU激活功能。

捷径路径：

- CONV2D具有形状为（1,1）的$F_3$个滤波器和（s，s）的步幅。其填充为“valid”。使用0作为“glorot_uniform”种子“kernel_initializer = initializer（seed = 0）”的值。

- BatchNorm对“channels”轴进行归一化。

最后一步：

- 捷径和主路径的值相加。

- 然后应用ReLU激活函数。这没有超参数。
  
### 练习2 - 卷积块

实现卷积块。主路径的第一个组件已经实现了; 现在轮到你来实现剩下的部分了！与之前一样，始终使用0作为随机初始化的种子，以确保与评分器的一致性。

- [Conv2D]（https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D）

- [BatchNormalization]（https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization）（轴：整数，应该规范化的轴（通常是特征轴））`BatchNormalization（axis = 3）（X，training = training）`。如果将training设置为False，则其权重不会随新示例更新。即在预测模式下使用模型时。

- 对于激活，请使用：`Activation（'relu'）（X）`

- [Add]（https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add）

我们向我们的函数添加了initializer参数。此参数接收像[tensorflow.keras.initializers]（https://www.tensorflow.org/api_docs/python/tf/keras/initializers）中包含的初始化程序函数或任何其他自定义初始化程序一样的初始化程序函数。默认情况下，它将设置为[random_uniform]（https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomUniform）

请记住，这些函数接受一个“seed”参数，其值可以是任何你想要的值，但在此笔记本中必须将其设置为0以进行分级目的。

In [ ]:
# UNQ_C2
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (s, s), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)
#X = np.random.randn(3, 4, 4, 6).astype(np.float32)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A = convolutional_block(X, f = 2, filters = [2, 4, 6], training=False)

assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

B = convolutional_block(X, f = 2, filters = [2, 4, 6], training=True)
assert np.allclose(B.numpy(), convolutional_block_output2), "Wrong values when training=True."

print('\033[92mAll tests passed!')


**Expected value**

```
tf.Tensor(
[[[0.         0.66683817 0.         0.         0.88853896 0.5274254 ]
  [0.         0.65053666 0.         0.         0.89592844 0.49965227]]

 [[0.         0.6312079  0.         0.         0.8636247  0.47643146]
  [0.         0.5688321  0.         0.         0.85534114 0.41709304]]], shape=(2, 2, 6), dtype=float32)
```

## 4 - 构建您的第一个ResNet模型（50层）

您现在拥有构建非常深的ResNet所需的必要块。下图详细描述了此神经网络的架构。图中的“ID BLOCK”代表“Identity block”，而“ID BLOCK x3”表示您应该将3个identity block叠加在一起。

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption> <center> <u> <font color ='purple'> <b>图5 </b> </u> <font color ='purple'>：<b> ResNet-50模型</ b> </ center> </ caption>

这个ResNet-50模型的细节如下：

- 零填充将输入填充到(3,3)。

- 阶段1:

- 2D卷积有64个形状为(7,7)的滤波器，使用步幅为(2,2)。

- BatchNorm应用于输入的“通道”轴。

- MaxPooling使用(3,3)窗口和(2,2)步幅。

- 阶段2:

- 卷积块使用大小为[64,64,256]的三组滤波器，“f”为3，“s”为1。

- 2个恒等块使用大小为[64,64,256]的三组滤波器，“f”为3。

- 阶段3:

- 卷积块使用大小为[128,128,512]的三组滤波器，“f”为3，“s”为2。

- 3个恒等块使用大小为[128,128,512]的三组滤波器，“f”为3。

- 阶段4:

- 卷积块使用大小为[256,256,1024]的三组滤波器，“f”为3，“s”为2。

- 5个恒等块使用大小为[256,256,1024]的三组滤波器，“f”为3。

- 阶段5:

- 卷积块使用大小为[512,512,2048]的三组滤波器，“f”为3，“s”为2。

- 2个恒等块使用大小为[512,512,2048]的三组滤波器，“f”为3。

- 2D平均池化使用形状为(2,2)的窗口。

- “平展(flatten)”层没有任何超参数。

- 全连接(Dense)层使用softmax激活将输入降到类的数量。
    


<a name='ex-3'></a>      
### 练习3-ResNet50

实现上面图中描述的50层ResNet。我们已经实现了阶段1和阶段2。请实现其余部分。（实现阶段3-5的语法应该与阶段2的相似）确保您遵循上面的命名约定。

您将需要使用此函数：

- 平均池化 [参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D)

以下是我们在下面的代码中使用的其他函数：

- Conv2D：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

- BatchNorm：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)（轴：整数，应归一化的轴（通常是特征轴））

- 零填充：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D)

- 最大池化：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

- 全连接层：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

- 加法：[参见参考文献](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add)

In [ ]:
# UNQ_C3
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ### START CODE HERE
    # Use the instructions above in order to implement all of the Stages below
    # Make sure you don't miss adding any required parameter
    
    ## Stage 3 (≈4 lines)
    # `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    
    # the 3 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    
    # Stage 4 (≈6 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    
    # the 5 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5 (≈3 lines)
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    
    # the 2 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D()(X)"
    X = AveragePooling2D((2, 2), strides=(2, 2))(X)
    
    ### END CODE HERE

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

Run the following code to build the model's graph. If your implementation is incorrect, you'll know it by checking your accuracy when running `model.fit(...)` below.

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

In [ ]:
from outputs import ResNet50_summary

model = ResNet50(input_shape = (64, 64, 3), classes = 6)

comparator(summary(model), ResNet50_summary)


As shown in the Keras Tutorial Notebook, prior to training a model, you need to configure the learning process by compiling the model.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

The model is now ready to be trained. The only thing you need now is a dataset!

Let's load your old friend, the SIGNS dataset.

<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> <b>Figure 6</b> </u><font color='purple'>  : <b>SIGNS dataset</b> </center></caption>


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Run the following cell to train your model on 10 epochs with a batch size of 32. On a GPU, it should take less than 2 minutes. 

In [ ]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

**Expected Output**:

```
Epoch 1/10
34/34 [==============================] - 1s 34ms/step - loss: 1.9241 - accuracy: 0.4620
Epoch 2/10
34/34 [==============================] - 2s 57ms/step - loss: 0.6403 - accuracy: 0.7898
Epoch 3/10
34/34 [==============================] - 1s 24ms/step - loss: 0.3744 - accuracy: 0.8731
Epoch 4/10
34/34 [==============================] - 2s 44ms/step - loss: 0.2220 - accuracy: 0.9231
Epoch 5/10
34/34 [==============================] - 2s 57ms/step - loss: 0.1333 - accuracy: 0.9583
Epoch 6/10
34/34 [==============================] - 2s 52ms/step - loss: 0.2243 - accuracy: 0.9444
Epoch 7/10
34/34 [==============================] - 2s 48ms/step - loss: 0.2913 - accuracy: 0.9102
Epoch 8/10
34/34 [==============================] - 1s 30ms/step - loss: 0.2269 - accuracy: 0.9306
Epoch 9/10
34/34 [==============================] - 2s 46ms/step - loss: 0.1113 - accuracy: 0.9630
Epoch 10/10
34/34 [==============================] - 2s 57ms/step - loss: 0.0709 - accuracy: 0.9778
```

The exact values could not match, but don't worry about that. The important thing that you must see is that the loss value decreases, and the accuracy increases for the firsts 5 epochs.

Let's see how this model (trained on only two epochs) performs on the test set.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Expected Output**:

<table>
    <tr>
        <td>
            <b>Test Accuracy</b>
        </td>
        <td>
           >0.80
        </td>
    </tr>

</table>

For the purposes of this assignment, you've been asked to train the model for just two epochs. You can see that it performs pretty poorly, but that's ok! The online grader will only run your code for a small number of epochs as well. Please go ahead and submit your assignment as is. 

After you have finished this official (graded) part of this assignment, you can also optionally train the ResNet for more iterations, if you want. It tends to get much better performance when trained for ~20 epochs, but this does take more than an hour when training on a CPU. 

Using a GPU, this ResNet50 model's weights were trained on the SIGNS dataset. You can load and run the trained model on the test set in the cells below. It may take ≈1min to load the model. Have fun! 

In [ ]:
pre_trained_model = tf.keras.models.load_model('resnet50.h5')

In [ ]:
preds = pre_trained_model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Congratulations** on finishing this assignment! You've now implemented a state-of-the-art image classification system! Woo hoo! 

ResNet50 is a powerful model for image classification when it's trained for an adequate number of iterations. Hopefully, from this point, you can use what you've learned and apply it to your own classification problem to perform state-of-the-art accuracy.

<font color = 'blue'>

**What you should remember**:

- Very deep "plain" networks don't work in practice because vanishing gradients make them hard to train.  
- Skip connections help address the Vanishing Gradient problem. They also make it easy for a ResNet block to learn an identity function. 
- There are two main types of blocks: The **identity block** and the **convolutional block**. 
- Very deep Residual Networks are built by stacking these blocks together.

<a name='5'></a>  
## 5 - Test on Your Own Image (Optional/Ungraded)

If you wish, you can also take a picture of your own hand and see the output of the model. To do this:
    1. Click on "File" in the upper bar of this notebook, then click "Open" to go on your Coursera Hub.
    2. Add your image to this Jupyter Notebook's directory, in the "images" folder
    3. Write your image's name in the following code
    4. Run the code and check if the algorithm is right! 

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = pre_trained_model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))


You can also print a summary of your model by running the following code.

In [ ]:
pre_trained_model.summary()

<a name='6'></a>  
## 6 - Bibliography

This notebook presents the ResNet algorithm from He et al. (2015). The implementation here also took significant inspiration and follows the structure given in the GitHub repository of Francois Chollet: 

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's GitHub repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
